In [0]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [115]:
text ="""Let us return for a moment to Lady Lovelace’s objection, which stated that the machine can only do what we tell it to do. 
One could say that a man can "inject" an idea into the machine, and that it will respond to a certain extent and then drop into quiescence, 
like a piano string struck by a hammer. Another simile would be an atomic pile of less than critical size: an injected idea is to correspond to a neutron 
entering the pile from without. Each such neutron will cause a certain disturbance which eventually dies away. If, however, the size of the pile is sufficiently 
increased, the disturbance caused by such an incoming neutron will very likely go on and on increasing until the whole pile is destroyed. Is there a corresponding 
phenomenon for minds, and is there one for machines? There does seem to be one for the human mind. The majority of them seem to be "sub critical," i.e. to
correspond in this analogy to piles of sub-critical size. An idea presented to such a mind will on average give rise to less than one idea in reply. A smallish 
proportion are supercritical. An idea presented to such a mind may give rise to a whole "theory" consisting of secondary, tertiary and more remote ideas.
Animals’ minds seem to be very definitely sub-critical. Adhering to this analogy we ask, "Can a machine be made to be super-critical? """
chars = list(set(text))
chars_dict = {c: i for i, c in enumerate(chars)}
print(len(text),len(chars))


1349 42


In [116]:
seq_len = 10
train_x= []
train_y = []
for i in range(0, len(text) - seq_len):
    x_str = text[i:i+seq_len]
    y_str = text[i+seq_len]
    x = [chars_dict[c] for c in x_str] 
    y = [chars_dict[c] for c in y_str]  
    train_x.append(x)
    train_y.append(y)
samples = len(train_x)
samples

1339

In [117]:
#reshape data_x in [samples,time steps, features]
train_x = np.reshape(train_x, (samples, seq_len, 1))
print(train_x.shape)
#one hot encoding
train_x = np_utils.to_categorical(train_x,num_classes = len(chars))
train_y = np_utils.to_categorical(train_y,num_classes = len(chars))
print(train_x.shape)

(1339, 10, 1)
(1339, 10, 42)


In [118]:

model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1], train_y.shape[1]), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(train_y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(train_x, train_y, batch_size = 8,epochs=128)

Epoch 1/128
1339/1339 [==============================] - 15s 11ms/step - loss: 3.0936
Epoch 2/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.9993
Epoch 3/128
1339/1339 [==============================] - 10s 7ms/step - loss: 2.9619
Epoch 4/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.8921
Epoch 5/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.7896
Epoch 6/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.6713
Epoch 7/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.5893
Epoch 8/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.4971
Epoch 9/128
1339/1339 [==============================] - 10s 8ms/step - loss: 2.4072
Epoch 10/128
1339/1339 [==============================] - 10s 7ms/step - loss: 2.3133
Epoch 11/128
1339/1339 [==============================] - 10s 7ms/step - loss: 2.2030
Epoch 12/128
1339/1339 [==============================] - 10s 

In [119]:
#test dataset
def reshape(x_s):
  t_x = []
  x = [chars_dict[c] for c in x_s]
  t_x.append(x)
  #one hot encoding
  t_x = np_utils.to_categorical(t_x,num_classes = len(chars))
  return t_x

test_x = []
x_str = text[:seq_len]
test_x = reshape(x_str)
test_x.shape

(1, 10, 42)

In [120]:
#predictions
res = list(x_str) 
num_chars = 150
for i in range(num_chars):
  prediction = model.predict(test_x, verbose=0)
  index = np.argmax(prediction)
  pred_char = chars[index]
  res.append(pred_char)
  #print(x_str,'-',pred_char)
  l = list(x_str) 
  l.append(pred_char)
  x_str = ''.join(l)
  x_str = x_str[1:]
  test_x = reshape(x_str)
print("Actual text sample-->\n",text[:seq_len+(num_chars)])
print("\nGenerated text for the sample-->\n","".join(map(str,res)))

Actual text sample-->
 Let us return for a moment to Lady Lovelace’s objection, which stated that the machine can only do what we tell it to do. 
One could say that a man can "inject"

Generated text for the sample-->
 Let us return for a moment to Lady Lovelace’s objection, which stated that the machine, and that it will respond to a neutron 
entering the pile from without. E
